In [ ]:
# pip install -U dataprep

In [1]:
#pip install pyforest

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import pyforest

In [4]:
data = pd.read_excel('ANZ synthesised transaction dataset.xlsx')
data.head()

<IPython.core.display.Javascript object>

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


Inference -This dataset contains 3 months' worth of transactions for 100 customers which includes purchases, recurring transactions and salary transactions.

In [5]:
data.columns #We can observe that we have 22 columns to work with here 

Index(['status', 'card_present_flag', 'bpay_biller_code', 'account',
       'currency', 'long_lat', 'txn_description', 'merchant_id',
       'merchant_code', 'first_name', 'balance', 'date', 'gender', 'age',
       'merchant_suburb', 'merchant_state', 'extraction', 'amount',
       'transaction_id', 'country', 'customer_id', 'merchant_long_lat',
       'movement'],
      dtype='object')

In [6]:
data.dtypes 

status                       object
card_present_flag           float64
bpay_biller_code             object
account                      object
currency                     object
long_lat                     object
txn_description              object
merchant_id                  object
merchant_code               float64
first_name                   object
balance                     float64
date                 datetime64[ns]
gender                       object
age                           int64
merchant_suburb              object
merchant_state               object
extraction                   object
amount                      float64
transaction_id               object
country                      object
customer_id                  object
merchant_long_lat            object
movement                     object
dtype: object

In [7]:
print("Data shape: ", data.shape)

Data shape:  (12043, 23)


In [ ]:
from dataprep.eda import plot 
plot(data)

# 2.1 Unique customers, unique transactions and 3 months of data
Before we explore explore each column individually, let's check for the following:

100 unique customers (customer_id)
Each row corresponds to a unique transaction ID (transaction_id)
3 months worth of data (date)

In [ ]:
print("Number of unique customer ID's: ", data.customer_id.nunique())

Inference-We do indeed have 100 unique customers in this dataset.

In [ ]:
print("Number of rows in dataset: ", len(data))
print("Number of unique transaction ID's: ", data.transaction_id.nunique())

Inference - The number of unique transaction ID's correspond to the number of rows in the dataset which suggests that each row represent a single, unique transaction.

In [ ]:
data.date.describe()

In [ ]:
pd.date_range(start = "2018-08-01", end = "2018-10-31").difference(data.date)

Inference - We do indeed have 3 months worth of data, starting from 1/8/2018 to 31/10/2018. However, there are only 91 unique days which means that there is one missing  and the missing date is '2018-08-16'

# 2.2 Non-null count and data types and Missing value Analysis 


In [ ]:
data.info()

In [ ]:
missing = data.isnull().sum()
missing = missing[missing > 0]
missing_percentage = round(missing / len(data), 3) * 100
pd.DataFrame({"Number of missing values": missing, "Percentage": missing_percentage}).sort_values(by = "Percentage", ascending = False)

Inference -Both bpay_biller_code and merchant_code columns contain a significant number of missing values. We will deal with those missing values later.

Let's examine the summary statistics of the numerical columns in the dataset.

# 2.3 Descriptive statistics of numerical variables¶

In [ ]:
data.describe()

Inference - The describe function is a good way to observe any anomalies in the numerical columns for example, we can check if the age and amount columns contain any negative values, or if a customer has an age of 150 etc. Here, I do not see any anomalies.

It also provides an overview of some basic statistics for the amount column. However, we do not know the breakdown of these transaction amounts for example, what sort of transactions are they, who made them etc. We will explore this as we proceed with our further analysis.

We can see see a huge spread between the maximum and minimum amount in the balance and amount columns. We will look into this further.

# 3. Exploratory data analysis (EDA) 
- Analysis of all the columns to check their information and draw certain insights from these columns . 

In [ ]:
from dataprep.eda import create_report 
report=create_report(data)
report.show()

# Categorical Variable analysis 
Here, we will skip the following columns as they are merely identifiers and won't provide us with much information and we will analyse all the other categporical variables so we can draw some insights from the data:

- bpay_biller_code
- account
- merchant_id
- merchant_code
- first_name
- transaction_id
- customer_id

#### 3.1.1 Status of transaction 

In [ ]:
data.status.value_counts(dropna = False)

Inference -Status of of the individual transactions: authorised means transaction has already been approved, posted means still in the process.

#### 3.1.2 Card present flag

In [ ]:
data.card_present_flag.value_counts(dropna = False)

Inference - We can see here that 
Card-not-present (CNP) transaction occurs when neither the cardholder nor the credit card is physically present at the time of the transaction. It's most common for orders that happen remotely over the phone, internet or mail.

#### 3.1.3 Currency of transaction

In [ ]:
data.currency.value_counts(dropna = False)

Inference - All transactions are made in AUD. Can potentially drop this column because unnecessary since it does not provide us with any additional information.

#### 3.1.4 Longitude and latitude

In [ ]:
data.long_lat.head()

Inference -
These are the coordinates where the transactions were made

#### 3.1.5 Transaction description (types of transactions)

In [ ]:
data.txn_description.value_counts(dropna = False)

In [ ]:
plt.figure(figsize = (10, 5))
sns.countplot(data.txn_description)
plt.title("Number of transactions by category")

Inference - We can validate the purchases in detail as we proceed . These are the descriptions for each transaction. Transactions mostly consist of sales payment.

Also, this might explain the missing values in the merchant columns as not all transactions are purchases of goods and services from merchants.

Lets analyse the transaction vs Amount 

In [ ]:
data[["txn_description", "amount"]].groupby("txn_description", as_index = False).mean().sort_values(by = "amount", ascending = False)

In [ ]:
plt.figure(figsize = (8, 5))
sns.barplot(x = "txn_description", y = "amount", data = data)
plt.title("Average transaction volume by category")

Inference - Pay/salary has the largest average transaction volume. Seems logical.

#### 3.1.6 Gender

In [ ]:
data.gender.value_counts(dropna = False)

In [ ]:
plt.figure(figsize = (5, 5))
sns.countplot(data.gender)
plt.title("Number of transactions by gender")

Inference-There are more male customer transactions than there are female.

In [ ]:
data[['gender','amount']].groupby("gender", as_index=False).mean().sort_values(by='amount',ascending=False)

In [ ]:
plt.figure(figsize = (5, 5))
sns.barplot(x = "gender", y = "amount", data = data)
plt.title("Average transaction volume by gender")

Inference- Male has a higher average transaction volume.

#### 3.1.7 Merchant suburb

In [ ]:
data.merchant_state.value_counts(dropna = False)

#### 3.1.8 Merchant State 

In [ ]:
data['merchant_state'].dropna()

In [ ]:
plt.figure(figsize = (12, 7))
sns.countplot(data.merchant_state)
plt.title("Number of transactions by state")

Inference - Similar to merchant suburb, these are the states where the transactions were made.

NSW and VIC are the top 2 states in number of transactions. (New South Wales and Victoria )

In [ ]:
data[["merchant_state", "amount"]].groupby("merchant_state", as_index = False).mean().sort_values(by = "amount", ascending = False)

In [ ]:
plt.figure(figsize = (12, 7))
sns.barplot(x = "merchant_state", y = "amount", data = data)
plt.title("Average transaction volume by state")

Inference - ACT has the highest average transaction volume but the variance is quite large. This indicates which state not only made more transactions but the amounts they spend during their transactions. 

#### 3.1.9 Extraction

In [ ]:
data.extraction.head()

Inference - This appears to be the timestamp for each transaction. We can cross-reference this with the date column.

In [ ]:
data[["date", "extraction"]].head()

Inference - Since we already have an existing date column, we can potentially extract only the time component out of the extraction column.

#### 3.1.10 Country

In [ ]:
data.country.value_counts(dropna = False)

Inference - 
All transactions were recorded within Australia. Therefore, can consider dropping this column since it does not provide us with any information.

#### 3.1.11 Merchant longitude and latitude

In [ ]:
data.merchant_long_lat.head()

inference - These are the coordinates of the merchants' location.

#### 3.1.12 Movement (debit/credit)

In [ ]:
data.movement.value_counts(dropna = False)

In [ ]:
plt.figure(figsize = (5 ,5))
sns.countplot(data.movement)
plt.title("Number of transactions by movement")

Infernce - Most transactions are overwhelmingly debit transactions.

In [ ]:
plt.figure(figsize = (5, 5))
sns.barplot(x = "movement", y = "amount", data = data)
plt.title("Average transaction volume by movement")

Inference - On the basis of amount most of the transactions were made by credit card . 

# 3.2 Numerical variables 
- Now we will analyse the numerical variables as we move forward . 

#### 3.2.1 Balance

In [ ]:
plt.figure(figsize = (8, 5))
sns.distplot(data.balance)
plt.title("Balance distribution")

Inference - We get a right skew distribution 

In [ ]:
plt.figure(figsize = (12, 4))
sns.boxplot(data.balance)

Inference - We can see that Distribution of the balance feature has a long tail.

#### 3.2.2 Age of customers

In [ ]:
plt.figure(figsize = (8, 5))
sns.distplot(data.age)
plt.title("Age distribution")

Inference - In this bank Majority of the customers belong in the 20-25 age bracket. Which also means they could hold zero balance accounts as students and could be working class too . 

#### 3.2.3 Transaction amount

In [ ]:
plt.figure(figsize = (8, 5))
sns.distplot(data.amount)
plt.title("Amount distribution")

# 4. Feature engineering

In this section, we are going to focus on dropping unwanted columns, deal with missing values and finally create some additional features that will make our data analysis more detailed and robust.

#### 4.1 Drop unwanted columns

In [ ]:
from dataprep.eda import plot_missing
plot_missing(data)

In [ ]:
print("Before: ", data.shape)
data = data.drop(["currency", "country"], axis = 1)
print("After: ", data.shape)


Inference - We drop the currency and column columns as they do not  provide us with new useful information . 

#### 4.2 Deal with missing values

In [ ]:
pd.DataFrame({"Number of missing values": missing, "Percentage": missing_percentage}).sort_values(by = "Percentage", ascending = False)

Let's investigate if there is a reason behind these missing values.

Recall that not all transactions are purchases-related, could this imply the missing values in the merchant columns?

In [ ]:
nonSales = data.loc[(data.txn_description != "SALES-POS") | (data.txn_description != "POS"), :]
nonSales.isnull().sum().sort_values(ascending = False)

As expected, missing values in the following columns all originate from non-purchases transactions: Let us proceed by filling these n/a values 

- card_present_flag
- merchant_state
- merchant_suburb
- merchant_id
- merchant_long_lat

In [ ]:
cols = ["card_present_flag", "merchant_state", "merchant_suburb", "merchant_id", "merchant_long_lat"]
for col in cols:
    data[col].fillna("n/a", inplace = True)

In [ ]:
missing = data.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(ascending = False)
missing_percentage = round(missing / len(data), 3) * 100
pd.DataFrame({"Number of missing values": missing, "Percentage": missing_percentage}).sort_values(by = "Percentage", ascending = False)

Both merchant_code and bpay_biller_code are severely missing, around 93%. Therefore, we will drop those 2 columns from the dataframe.

In [ ]:
data = data.drop(["merchant_code", "bpay_biller_code"], axis = 1)

In [ ]:
data.isnull().sum().max()

### 4.3 Create features for month, dayofweek and hour

In [ ]:
daily_amount = pd.DataFrame(data.groupby("date").amount.sum())
daily_amount.head()

Lets do a time series analysis after creating our dataset to analyse this data - 

In [ ]:
fig, ax = plt.subplots(figsize = (12, 5))
ax.plot(daily_amount.index, daily_amount.amount)
plt.title("Transaction volume from 1/8/2018 to 31/10/2018")
plt.xlabel("Date")
plt.ylabel("Transaction volume")

There appears to be a cyclical pattern in transaction volume over the 3 month period. My hypothesis for this occurence is:

Salaries are paid out only on certain days of the week
People tend to spend more during the weekends (shopping, going out with friends etc)
Therefore, to motivate this analysis, we will create 3 new features from both the date and extraction columns:

- month
- dayofweek
- hour

In [ ]:
data["month"] = pd.DatetimeIndex(data.date).month
data["dayofweek"] = pd.DatetimeIndex(data.date).dayofweek
data[["date", "month", "dayofweek"]].head()

Inference-In addition to month and day of week, we can also get the hour in which those transactions took place from the extraction column.

First, I will update the extraction column such that it only contains the time. Then, I will create a new hour column which contains only the hour component of each transaction.

In [ ]:
data.extraction.head()

In [ ]:
data["extraction"] = [timestamp.split("T")[1].split(".")[0] for timestamp in data.extraction]
data.extraction.head()

In [ ]:
data["hour"] = [time.split(":")[0] for time in data.extraction]
data[["extraction", "hour"]].head()

In [ ]:
print("Before: ", data.hour.dtype)
data["hour"] = pd.to_numeric(data.hour)
print("After: ", data.hour.dtype)

Now that our dataframe is in a much better shape and we have new features to work with, let's now do a deep dive and see if we can gather any interesting insights about customers' transactional behaviour.

In [ ]:
data.head()

In [ ]:
from dataprep.eda import plot_correlation 
plot_correlation(data)

In [ ]:
from  dataprep.eda import plot_correlation 
plot_correlation(data)

# 5.1 Purchases amount and overall amount


In [ ]:
amt= data['amount']
avg_trans_amt = amt.sum()/ amt.count()
print(avg_trans_amt)

In [ ]:
data.columns

In [ ]:
data.nunique()

In [ ]:
#This amount includes only Pos and Sales Pos as these are purchase amounts we are dealing with here . 
purchases_amount = data.loc[(data.txn_description == "POS") | (data.txn_description == "SALES-POS"), "amount"]
purchases_amount.head()

In [ ]:
plt.figure(figsize = (8, 5))
sns.distplot(purchases_amount)
plt.title("Purchase transaction amount")

In [ ]:
purchases_amount.describe()

In [ ]:
plt.figure(figsize = (8, 5))
sns.distplot(data.amount)
plt.title("Overall transaction amount")

In [ ]:
data.amount.describe()

Inference-Variance in overall transaction amount is much higher than purchases transaction amount due to the inclusion of salary payments.

# 5.2 Transaction volume per customer

In [ ]:
customer_monthly_volume = pd.DataFrame(data.groupby("customer_id").amount.sum() / 3) 
customer_monthly_volume.head()

In [ ]:
plt.figure(figsize = (12, 5))
sns.distplot(customer_monthly_volume.amount)
plt.title("Customers' monthly transaction volume")

# 5.3 Transaction volume over time

In [ ]:
plt.figure(figsize = (5, 5))
sns.barplot(x = "month", y = "amount", data = data)
plt.title("Average transaction volume by month")

Inference - It looks like October month has most number of transactions made by volume . 

In [ ]:
average_daily_volume = pd.DataFrame(data.groupby("dayofweek").amount.mean())
average_daily_volume.head()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 5))
ax.plot(average_daily_volume.index, average_daily_volume.amount)
plt.title("Average transaction volume per day")
plt.ylabel("Transaction volume")
plt.xlabel("Day of week")


Inference - We can see that Friday and Saturday have the lowest average transaction volume. Let's see a breakdown of these transactions by types of transactions.

To simplify the categories, I will group all transactions into 3 categories:

- Salary
- Purchase
- Others

In [ ]:
#Lets do a transaction Type Further analysis to proceed 
data.txn_description.value_counts()

In [ ]:
#Data Frame filtering to find the categories that we can group together 
data.loc[data.txn_description == "PAY/SALARY", "category"] = "Salary"
data.loc[(data.txn_description == "SALES-POS") | (data.txn_description == "POS"), "category"] = "Purchase"
data.category.fillna("Others", inplace = True)
data[["txn_description", "category"]].head(10)

In [ ]:
stacked_barplot = pd.DataFrame(data.groupby(["dayofweek", "category"]).amount.count())
stacked_barplot.unstack().plot(kind = "bar", stacked = True, figsize = (12, 7))
plt.title("Number of transactions each day by category")
plt.legend(["Others", "Purchases", "Salaries"])
plt.ylabel("Number of transactions")
plt.xlabel("Day of week")

Inference - From above we can see that No salaries were paid on Friday and Saturday, therefore lower average transaction volume on those days.

Also, it does not seem to appear that people spend more on purchases over the weekends.

In [ ]:
average_hourly_volume = pd.DataFrame(data.groupby("hour").amount.mean())
average_hourly_volume.head()

In [ ]:
fig, ax = plt.subplots(figsize = (8, 5))
ax.plot(average_hourly_volume.index, average_hourly_volume.amount)
plt.title("Average transaction volume per hour")
plt.ylabel("Transaction volume")
plt.xlabel("Hour")

Inference - When we analyse the transaction on the basis of hours we can see that Highest transaction volumes happen during the midday and late afternoon. Let's see a breakdown of these transactions by categories.

In [ ]:
stacked_barplot = pd.DataFrame(data.groupby(["hour", "category"]).amount.count())
stacked_barplot.unstack().plot(kind = "bar", stacked = True, figsize = (12, 7))
plt.title("Number of transactions each hour by category")
plt.legend(["Others", "Purchases", "Salaries"])
plt.ylabel("Number of transactions")
plt.xlabel("Hour")

# 5.4 Distance between customers and merchants-

We can explore further the  - haversine library in Python which calculates the distance between two points on the Earth using their latitude and longitude. We can obtain the latitude and longitude of both the customers and merchants from the following columns:

long_lat , 
merchant_long_lat

In [ ]:
#Creation of a final report for the same 

In [ ]:
data[["long_lat", "merchant_long_lat"]].head()

In [ ]:
from dataprep.eda import create_report 
report=create_report(data)

In [ ]:
report.show()

6. Save data

In [ ]:
data.to_pickle('C:/Users/Daniella/Forage ANZ/data.pkl')